In [23]:
import re
import nltk
from textblob import TextBlob
from collections import Counter
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
from textblob.wordnet import VERB
from nltk.tokenize import RegexpTokenizer

# 
# BW: This script can be used to extract (possible) uknown concepts from a transcript with the use of the Textblob package
#

detokenizer= Detok()
speaker_words = {}
speaker_pattern = re.compile(r'^(\w+?):(.*)$')

ontologyList = ['po','approval', 'matching' , 'voucher' , 'VM', 'AP', 'PO' , 'address' ,'book', 'ledger', 'procurement' , 'currency', 'file-upload' ,'invoice', 'OCR', 'ocr', 'jd', 'JD', 'payment', 'qa' , 'QA' , 'workflow'] 

stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['Yeah','yeah', 'Um','Well','Right', 'I', 'Yes','Uh','uh','Okay','okay', 'Sorry','sorry','Yep','yep','Uhh','uhh','Gonna', 'gonna','Gon na','gon na','Mm','mm','Wanna','Wan na','wanna','wan na','Ok','ok','OK','Oh','oh','Mhm','mhm','Sure','sure','Mhmm','mhmm','Jeez','jeez','Hi','hi','wow','Wow','True','true'])

with open("transcript-splitted/.txt", "r") as f:
        lines = f.readlines()
        current_speaker = None
        for line in lines:
                line = line.strip()
                match = speaker_pattern.match(line)
                if match is not None:
                        current_speaker = match.group(1)
                        line = match.group(2).strip()
                        if current_speaker not in speaker_words.keys():
                                speaker_words[current_speaker] = []
                if current_speaker:
                        words = [word.strip() for word in line.split(' ') if len(word.strip()) > 0]
                        speaker_words[current_speaker].extend(words)

dictget = lambda d, *k: [d[i] for i in k]
#Add speakers here based on the total amount of speakers found in text
spk_0 = dictget(speaker_words, 'spk_0')
tokenizer = RegexpTokenizer(r'\w+')
#Filter stopwords from text and filter ontology words from text
#Copy lines 43-64 for each speaker and change variable names accordingly
text_0 = detokenizer.detokenize(spk_0)
spk_0_tokenized = tokenizer.tokenize(text_0)
filtered_words_0 = [word for word in spk_0_tokenized if word not in stopwords]
new_text_0 = detokenizer.detokenize(filtered_words_0)
for word in ontologyList:
    new_text_0 = new_text_0.replace(word, "")
blob_0 = TextBlob(new_text_0)
#Get noun phrases from remaining text 
noun_list_0 = []
for nouns in blob_0.noun_phrases:
    noun_list_0.append(nouns)

def removeElements(lst, k): 
    counted_0 = Counter(lst) 
    return [el for el in lst if counted_0[el] >= k]
#Remove concepts that have a frequency that is less than the second parameter
removed_0=removeElements(noun_list_0, 1)
countedFinished_0 = Counter(removed_0)
#Put resulting concepts in Dataframe
df_0 = pd.DataFrame.from_dict(countedFinished_0, orient='index').reset_index()
df_0.columns = ['Noun Phrase', 'Frequency']
df_0.insert(2, "Speaker", "spk_0", True) 
df_new_0 = df_0.sort_values(by=['Frequency'], ascending=False)
#Combine dataframes based on the amount of speakers
frames = [df_new_0]
result = pd.concat(frames)
result_sorted = result.sort_values(by=['Frequency'], ascending=False)
result_sorted.to_excel("output/.xlsx", index=False)